In [1]:
import torch
torch.set_num_threads(1)  #Number of threads from the CPU core to be used
from torch import nn, Tensor
import tensorflow_datasets as tfds  # import tensorflow dataset 
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

import matplotlib.pyplot as plt

In [2]:
from torch.nn.utils.rnn import pad_sequence
import torch

# Suppose you have a list of tensors representing sequences
sequences = [torch.tensor([1, 2, 3]), torch.tensor([4, 5]), torch.tensor([6, 7, 8, 9])]

# Pad the sequences to make them equal in length
padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)

print(padded_sequences)

tensor([[1, 2, 3, 0],
        [4, 5, 0, 0],
        [6, 7, 8, 9]])


In [5]:
data_dir = "../../tensorflow_datasets/"
dataset = "clic_edm_qq_pf"

# Load dataset
builder = tfds.builder(dataset, data_dir=data_dir)

In [8]:
X_FEATURES_TRK= [ 
    "elemtype",
    "pt",
    "eta",
    "sin_phi",
    "cos_phi",
    "p",
    "chi2",
    "ndf",
    "dEdx",
    "dEdxError",
    "radiusofInnermostHit",
    "tanLambda",
    "D0",
    "omega",
    "Z0",
    "time",]

X_FEATURES_CL = [
    "elemtype",
    "et",
    "eta",
    "sin_phi",
    "cos_phi",
    "energy",
    "position.x",
    "position.y",
    "position.z",
    "iTheta",
    "energy_ecal",
    "energy_hcal",
    "energy_other",
    "num_hits",
    "sigma_x",
    "sigma_y",
    "sigma_z",
]

Y_FEATURES = ["cls_id", "charge", "pt", "eta", "sin_phi", "cos_phi", "energy"]
Y_CLASSES = [0, 211, 130, 22, 11, 13]

INPUT_DIM = max(len(X_FEATURES_TRK), len(X_FEATURES_CL))
NUM_CLASSES = len(Y_CLASSES)

# Model

In [18]:
class FocalLoss(nn.Module):
    def __init__(
        slef, alpha= None, gamma=0.0, reduction="mean", ignore_index= -100):
            
            if reduction not in ("mean", "sum", "none"):
                raise ValueError('Reduction must be one of: "mean", "sum", "none".')
                
            super().__init__()
            self.alpha = alpha
            self.gamma = gamma
            self.reduction = reduction
            
            self.null_loss = nn.NLLLoss(weight=alpha, reduction="none")
            
    def __repr_(self):
        arg_keys = ["alpha", "gamma", "reduction"]
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f"{k}={v!r}" for k, v in zip(arg_keys, arg_vals)]
        arg_str = ", ".join(arg_strs)
        return f"{type(self).__name__}({arg_str})"
    
    def forward(self, x:Tensor, y:Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ...., dK) --> (N * d1 *... *dK, C)
            C = x.shape[1]
            x = x.\\